# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

поиграться с параметрами и архитектурой - количеством каналов (размерностью эмбеддинга), глубиной нейросети, силой Dropout, добавить BatchNorm или другую нормализацию
подключить прореженные (dilated) свёртки, чтобы увеличить рецептивное поле без увеличения числа параметров
добавить взвешивание классов
использовать в качестве обозначения начала и конца слова не 0, а какой-нибудь другой токен (для 0 nn.Embedding всегда выдаёт нулевой вектор, а в этом случае для начала а конца слова будут учиться специальные вектора)

sample_data


In [0]:
# Если Вы запускаете ноутбук на colab,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git
import sys; sys.path.append('/content/stepik-dl-nlp')

fatal: destination path 'stepik-dl-nlp' already exists and is not an empty directory.


In [0]:
!pip install pyconll
!pip install spacy-udpipe 

In [0]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
!wget -O /content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O /content/stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
full_train = pyconll.load_from_file('/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu')

In [0]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [0]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [0]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [0]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 150
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [0]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [0]:
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [0]:
train_inputs[1][:5]

tensor([[ 0, 39,  4, 25,  4, 11, 20,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 19,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 19, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0

In [0]:
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

In [0]:

class_weigts = np.unique(train_labels, return_counts= True)[1]/np.unique(train_labels, return_counts= True)[1].sum()

## Вспомогательная свёрточная архитектура

In [0]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [0]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [0]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47826


In [0]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 501 итераций, 292.63 сек
Среднее значение функции потерь на обучении 0.07983567332941853
Среднее значение функции потерь на валидации 0.03931898243799068
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 292.58 сек
Среднее значение функции потерь на обучении 0.02960396786694398
Среднее значение функции потерь на валидации 0.028264460310635
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 292.59 сек
Среднее значение функции потерь на обучении 0.02479359470382184
Среднее значение функции потерь на валидации 0.025489307294535166
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 292.59 сек
Среднее значение функции потерь на обучении 0.02264059107088876
Среднее значение функции потерь на валидации 0.02291780728662368
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 292.57 сек
Среднее значение функции потерь на обучении 0.02144256806718613
Среднее значение функции потерь на валидации 0.02221147347204756
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 292.59 сек
Средне

In [0]:
#!ls /content/stepik-dl-nlp/models/single_token_pos.pth -la
from google.colab import files
files.download('/content/stepik-dl-nlp/models/single_token_pos.pth')

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), '/content/stepik-dl-nlp/models/single_token_pos.pth')

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
#single_token_model.load_state_dict(torch.load('/content/stepik-dl-nlp/models/single_token_pos.pth'))

In [0]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:22, 68.08it/s]                               


Среднее значение функции потерь на обучении 0.016120733693242073


  3%|▎         | 7/205.75 [00:00<00:03, 62.54it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.88      0.94      0.91     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.88      0.90      0.89     44101
         AUX       0.88      0.50      0.64      7522
       CCONJ       0.88      0.98      0.93     30432
         DET       0.83      0.84      0.84     21968
        INTJ       1.00      0.10      0.19        78
        NOUN       0.98      0.92      0.95    214497
         NUM       0.96      0.94      0.95     13746
        PART       0.97      0.77      0.86     26651
        PRON       0.88      0.87      0.88     38438
       PROPN       0.78      0.97      0.86     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.84      0.86      0.85     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.90      0.95      0.93     97670
           X       0.94    

206it [00:03, 67.78it/s]                            


Среднее значение функции потерь на валидации 0.018282104283571243
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.87      0.94      0.90     11222
         ADP       1.00      0.99      1.00     10585
         ADV       0.89      0.90      0.89      6165
         AUX       0.87      0.51      0.64      1106
       CCONJ       0.89      0.98      0.93      4410
         DET       0.80      0.83      0.82      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.98      0.91      0.94     27974
         NUM       0.95      0.92      0.93      1829
        PART       0.96      0.78      0.86      3877
        PRON       0.88      0.86      0.87      5598
       PROPN       0.77      0.95      0.85      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.82      0.86      0.84      2258
         SYM       1.00      1.00      1.00        53
        VERB   

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [0]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [0]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84882


In [0]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Эпоха 0
Эпоха: 501 итераций, 300.31 сек
Среднее значение функции потерь на обучении 0.07253477907555546
Среднее значение функции потерь на валидации 0.026585261678636663
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 300.30 сек
Среднее значение функции потерь на обучении 0.02449367340364142
Среднее значение функции потерь на валидации 0.019625423383889812
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 300.30 сек
Среднее значение функции потерь на обучении 0.01987817867876527
Среднее значение функции потерь на валидации 0.016515175046602097
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 300.31 сек
Среднее значение функции потерь на обучении 0.017681164724637054
Среднее значение функции потерь на валидации 0.014434890411240926
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 300.30 сек
Среднее значение функции потерь на обучении 0.016505230773991336
Среднее значение функции потерь на валидации 0.013584769971779372
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 300.30 се

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), '/content/stepik-dl-nlp/models/sentence_level_pos.pth')
#files.download('/content/stepik-dl-nlp/models/sentence_level_pos.pth')

In [0]:
# Если Вы запускаете ноутбук на colab, добавьте в начало пути /content/stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('./models/sentence_level_pos.pth'))

In [0]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:23, 65.38it/s]                               


Среднее значение функции потерь на обучении 0.010000622831285


  3%|▎         | 7/205.75 [00:00<00:03, 62.42it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.92      0.95      0.94     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.89      0.93      0.91     44101
         AUX       0.92      0.84      0.88      7522
       CCONJ       0.94      0.98      0.96     30432
         DET       0.91      0.94      0.92     21968
        INTJ       0.88      0.27      0.41        78
        NOUN       0.98      0.96      0.97    214497
         NUM       0.97      0.94      0.95     13746
        PART       0.97      0.89      0.93     26651
        PRON       0.96      0.92      0.94     38438
       PROPN       0.93      0.97      0.95     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.89      0.83      0.85     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.94      0.97      0.96     97670
           X       0.75    

206it [00:03, 66.13it/s]                            


Среднее значение функции потерь на валидации 0.011332117021083832
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.92      0.94      0.93     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.88      0.92      0.90      6165
         AUX       0.91      0.81      0.86      1106
       CCONJ       0.94      0.98      0.96      4410
         DET       0.90      0.92      0.91      3085
        INTJ       0.60      0.27      0.37        11
        NOUN       0.97      0.96      0.97     27974
         NUM       0.95      0.92      0.94      1829
        PART       0.97      0.89      0.93      3877
        PRON       0.96      0.90      0.93      5598
       PROPN       0.91      0.95      0.93      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.86      0.81      0.83      2258
         SYM       1.00      0.96      0.98        53
        VERB   

## Применение полученных теггеров и сравнение

In [0]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [0]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [0]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 183.99it/s]                    

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-VERB мама-NOUN любит-VERB печь-VERB

ведро-ADV дало-VERB течь-VERB вода-NOUN стала-VERB течь-VERB

три-NUM да-PART три-NUM будет-VERB дырка-NOUN

три-NUM да-PART три-NUM будет-VERB шесть-NUM

сорок-NOUN сорок-NOUN



In [0]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 177.26it/s]                    

мама-NOUN мыла-VERB раму-ADV

косил-VERB косой-ADJ косой-ADJ косой-NOUN

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-VERB вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-VERB

сорок-NUM сорок-NOUN



## Свёрточный модуль своими руками

In [0]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [0]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      # single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      # context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=nn.Conv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=nn.Conv1d))

print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84882


In [0]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Досрочно остановлено пользователем


In [0]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:37, 40.62it/s]                               


Среднее значение функции потерь на обучении 0.009504403918981552


  2%|▏         | 4/205.75 [00:00<00:05, 37.18it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.94      0.94      0.94     85589
         ADP       1.00      1.00      1.00     81963
         ADV       0.92      0.92      0.92     44101
         AUX       0.89      0.94      0.91      7522
       CCONJ       0.93      0.98      0.96     30432
         DET       0.93      0.91      0.92     21968
        INTJ       0.33      0.06      0.11        78
        NOUN       0.98      0.97      0.97    214497
         NUM       0.96      0.96      0.96     13746
        PART       0.98      0.87      0.92     26651
        PRON       0.94      0.94      0.94     38438
       PROPN       0.95      0.96      0.95     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.87      0.97      0.92     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.95      0.97      0.96     97670
           X       0.90    

206it [00:05, 40.65it/s]                            


Среднее значение функции потерь на валидации 0.010808326303958893
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.93      0.92      0.93     11222
         ADP       1.00      1.00      1.00     10585
         ADV       0.92      0.91      0.91      6165
         AUX       0.88      0.93      0.90      1106
       CCONJ       0.94      0.99      0.96      4410
         DET       0.91      0.89      0.90      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.97      0.96      0.97     27974
         NUM       0.95      0.94      0.94      1829
        PART       0.98      0.88      0.92      3877
        PRON       0.93      0.93      0.93      5598
       PROPN       0.94      0.94      0.94      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.86      0.96      0.91      2258
         SYM       1.00      1.00      1.00        53
        VERB   

In [0]:
# поиграться с параметрами и архитектурой - количеством каналов (размерностью эмбеддинга), глубиной нейросети, силой Dropout, добавить BatchNorm или другую нормализацию
# подключить прореженные (dilated) свёртки, чтобы увеличить рецептивное поле без увеличения числа параметров
# добавить взвешивание классов
# использовать в качестве обозначения начала и конца слова не 0, а какой-нибудь другой токен (для 0 nn.Embedding всегда выдаёт нулевой вектор, а в этом случае для начала а конца слова будут учиться специальные вектора)

In [0]:
features_num = 49
kernel_size = 3
l = nn.Sequential(
  nn.Conv1d(features_num, features_num, kernel_size, padding=kernel_size//2, dilation=7),                
  nn.LeakyReLU()
)

l

Sequential(
  (0): Conv1d(49, 49, kernel_size=(3,), stride=(1,), padding=(1,), dilation=(7,))
  (1): LeakyReLU(negative_slope=0.01)
)

In [0]:
l[0].out_channels

49

#base line 0.val 89
embedding_size=96 val macro avg 0.91

layers_n=4  val macro avg 0.91

dropout=0.2  0.92

BatchNorm1d .9

BN + Dout=.2 0.89

layers_n=4+ dropout=0.2 0.92

layers_n=3+dout=.2+dilation 0.91

layers_n=4+dout=.2+dilation 0.91

last <PAD> 0.93

weights loss 0.93

weights loss + last <PAD> 0.93


In [0]:
2**2

4

In [0]:
#torch.backends.cudnn.deterministic=False #for dilation is too slow
torch.backends.cudnn.deterministic=False
torch.backends.cudnn.benchmark=False

class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        # print(tokens.shape) #64,205,49
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []

        # layers.append(nn.Sequential(
        #     conv_layer(features_num, features_num, kernel_size, padding=1, dilation=1),                
        #     nn.Dropout(dropout),                
        #     nn.LeakyReLU()))
        # layers.append(nn.Sequential(
        #     conv_layer(features_num, features_num, kernel_size, padding=2, dilation=2),                
        #     nn.Dropout(dropout),                
        #     nn.LeakyReLU()))

        for layer_n in range(layers_n):
            pad = 2**layer_n
            layers.append(nn.Sequential(
                # conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                conv_layer(features_num, features_num, kernel_size, dilation=pad, padding=pad),
                nn.Dropout(dropout),                
                nn.LeakyReLU()))

        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=49,
                                                      single_backbone_kwargs=dict(layers_n=4, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d),
                                                      context_backbone_kwargs=dict(layers_n=4, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d))

print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))


Количество параметров 66266
Эпоха 0
Эпоха: 501 итераций, 150.32 сек
Среднее значение функции потерь на обучении 0.0752971372241805
Среднее значение функции потерь на валидации 0.022371099920629866
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 150.29 сек
Среднее значение функции потерь на обучении 0.023852696632984987
Среднее значение функции потерь на валидации 0.016302825191454724
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 150.30 сек
Среднее значение функции потерь на обучении 0.019297824547943956
Среднее значение функции потерь на валидации 0.014493120460780246
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 150.34 сек
Среднее значение функции потерь на обучении 0.017380965356817265
Среднее значение функции потерь на валидации 0.012411150200716635
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 150.29 сек
Среднее значение функции потерь на обучении 0.016242713130222346
Среднее значение функции потерь на валидации 0.012203914924792133
Новая лучшая модель!

Эпоха 5
Э

In [0]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for layer_n in range(layers_n):
            layers.append(nn.Sequential(
                #conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                #conv_layer(features_num-layer_n, features_num-layer_n, kernel_size, padding=kernel_size//2, dilation=layer_n+1),                
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),                
                #nn.BatchNorm1d(features_num),
                nn.Dropout(dropout),                
                #nn.LeakyReLU()))
                nn.ReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [0]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=96,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=MyConv1d))
                                                      # single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d),
                                                      # context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d))

print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 182610


<generator object Module.parameters at 0x7f6946278c50>

In [0]:
#criterion = F.cross_entropy(sentence_level_model_my_conv.parameters(), weight=class_weigts)
weights = torch.tensor(class_weigts,dtype=torch.float).cuda()
criterion = nn.CrossEntropyLoss(weight=weights)

In [0]:

(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      criterion,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Эпоха: 501 итераций, 114.20 сек
Среднее значение функции потерь на обучении 0.01679234800528235
Среднее значение функции потерь на валидации 0.00042618350454347927
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 114.12 сек
Среднее значение функции потерь на обучении 0.00035831949042371044
Среднее значение функции потерь на валидации 0.00026132916313559187
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 114.12 сек
Среднее значение функции потерь на обучении 0.0002687701151393514
Среднее значение функции потерь на валидации 0.000209679032790032
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 114.07 сек
Среднее значение функции потерь на обучении 0.00022668232983875158
Среднее значение функции потерь на валидации 0.00020352268116352712
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 114.01 сек
Среднее значение функции потерь на обучении 0.0001980092564628863
Среднее значение функции потерь на валидации 0.00019072859565609845
Новая лучшая модель!

Эпоха 5
Эпоха: 501 ите

In [0]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:53, 28.31it/s]                               


Среднее значение функции потерь на обучении 0.01649223454296589


  1%|▏         | 3/205.75 [00:00<00:07, 25.80it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.88      0.94      0.91     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.90      0.78      0.84     44101
         AUX       0.91      0.42      0.57      7522
       CCONJ       0.94      0.95      0.95     30432
         DET       0.85      0.86      0.86     21968
        INTJ       0.00      0.00      0.00        78
        NOUN       0.93      0.98      0.96    214497
         NUM       0.95      0.80      0.87     13746
        PART       0.95      0.84      0.89     26651
        PRON       0.90      0.91      0.90     38438
       PROPN       0.96      0.88      0.92     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.88      0.76      0.81     16219
         SYM       0.00      0.00      0.00       840
        VERB       0.92      0.95      0.93     97670
           X       0.00    

206it [00:07, 28.45it/s]                            


Среднее значение функции потерь на валидации 0.018353989347815514
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.88      0.93      0.90     11222
         ADP       0.99      0.99      0.99     10585
         ADV       0.90      0.78      0.84      6165
         AUX       0.88      0.37      0.52      1106
       CCONJ       0.95      0.95      0.95      4410
         DET       0.84      0.85      0.85      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.93      0.98      0.95     27974
         NUM       0.95      0.75      0.84      1829
        PART       0.95      0.84      0.89      3877
        PRON       0.89      0.90      0.90      5598
       PROPN       0.94      0.87      0.91      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.87      0.77      0.81      2258
         SYM       0.00      0.00      0.00        53
        VERB   

In [0]:
weights_cpu = torch.tensor(class_weigts,dtype=torch.float)

train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels), weight=weights_cpu)
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels), weight=weights_cpu)
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:58, 26.01it/s]                               


Среднее значение функции потерь на обучении 9.381408744957298e-05


  1%|▏         | 3/205.75 [00:00<00:07, 25.42it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.90      0.94      0.92     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.85      0.83      0.84     44101
         AUX       0.88      0.32      0.47      7522
       CCONJ       0.89      0.98      0.93     30432
         DET       0.84      0.89      0.86     21968
        INTJ       0.00      0.00      0.00        78
        NOUN       0.93      0.98      0.96    214497
         NUM       0.94      0.85      0.89     13746
        PART       0.97      0.78      0.87     26651
        PRON       0.92      0.89      0.90     38438
       PROPN       0.98      0.82      0.89     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.91      0.64      0.75     16219
         SYM       0.00      0.00      0.00       840
        VERB       0.90      0.97      0.93     97670
           X       0.00    

206it [00:07, 28.43it/s]                            


Среднее значение функции потерь на валидации 0.0001075459731509909
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.90      0.93      0.91     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.84      0.82      0.83      6165
         AUX       0.85      0.29      0.43      1106
       CCONJ       0.90      0.98      0.94      4410
         DET       0.82      0.88      0.85      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.93      0.98      0.96     27974
         NUM       0.93      0.80      0.86      1829
        PART       0.97      0.79      0.87      3877
        PRON       0.91      0.88      0.90      5598
       PROPN       0.96      0.82      0.88      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.90      0.62      0.74      2258
         SYM       0.00      0.00      0.00        53
        VERB  

In [0]:
#train_loss = F.cross_entropy(torch.tensor(train_pred), torch.tensor(train_labels), weight=weights_cpu)
#      <NOTAG>       1.00      1.00      1.00   1231232
#          ADJ       0.90      0.93      0.91     11222
#          ADP       1.00      0.99      0.99     10585
#          ADV       0.84      0.82      0.83      6165
#          AUX       0.85      0.29      0.43      1106
#        CCONJ       0.90      0.98      0.94      4410
#          DET       0.82      0.88      0.85      3085
#         INTJ       0.00      0.00      0.00        11
#         NOUN       0.93      0.98      0.96     27974
#          NUM       0.93      0.80      0.86      1829
#         PART       0.97      0.79      0.87      3877
#         PRON       0.91      0.88      0.90      5598
#        PROPN       0.96      0.82      0.88      4438
#        PUNCT       1.00      1.00      1.00     22694
#        SCONJ       0.90      0.62      0.74      2258
#          SYM       0.00      0.00      0.00        53
#         VERB       0.89      0.97      0.92     13078
#            X       0.00      0.00      0.00       105

#     accuracy                           0.99   1349720
#    macro avg       0.77      0.71      0.73   1349720
# weighted avg       0.99      0.99      0.99   1349720

train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

In [0]:
class WeightedMultilabelCrossEntropy(nn.Module):  
    def __init__(self, weights: torch.Tensor):  
        super(WeightedMultilabelCrossEntropy, self).__init__()  
        self.cerition = F.cross_entropy #nn.BCEWithLogitsLoss(reduction='none')  
        self.weights = weights  
  
    def forward(self, outputs, targets):  
        loss = self.cerition(outputs, targets)  
        return (loss * self.weights).mean()    

def get_class_weights(labels):
  count_labels = np.unique(labels, return_counts= True)[1]
  return count_labels/count_labels.sum()
  #exclude 0 <PAD> weight
  # return count_labels[1:]/count_labels[1:].sum()

from sklearn.utils import class_weight
list_classes = np.arange(len(label2id))
y = train_labels.view(-1) #train_labels[list_classes].values
weights = class_weight.compute_sample_weight('balanced', y) #0.92
weights = torch.tensor(weights).to('cuda')
criterion = WeightedMultilabelCrossEntropy(weights) #0.93
#last <PAD> 0.93

# weights = get_class_weights(train_labels) #0.91
# weights = torch.tensor(weights).to('cuda')
# criterion = WeightedMultilabelCrossEntropy(weights) #0.93
#last <PAD> 0.92

In [0]:
import torch
from torch.autograd import Variable
from torch import nn
from copy import deepcopy
import numpy as np
torch.manual_seed(42)
np.set_printoptions(suppress=True)
print(torch.__version__)

img = Variable(torch.Tensor(1, 8, 8).uniform_(-1, 1).float()).cuda()

net = nn.Sequential(
  nn.Conv1d(8,8, kernel_size=3, padding=1, dilation=1, stride=1),
  nn.Conv1d(8,8, kernel_size=3, padding=2, dilation=2, stride=1), 
  nn.Conv1d(8,8, kernel_size=3, padding=4, dilation=4, stride=1) )
#conv1d = deepcopy(conv1).cuda().float()
net.cuda()
y1, y1d = img.clone(), img.clone()
for i in range(2):
    y1 = net(y1)
    print(y1)

1.3.1
tensor([[[-0.0050,  0.1567,  0.1345,  0.2130,  0.1372, -0.0005,  0.1036,
          -0.0470],
         [-0.0473,  0.0489, -0.0665, -0.0818, -0.0143, -0.0587, -0.1225,
          -0.2250],
         [-0.1128, -0.0416, -0.0944,  0.0580, -0.0914, -0.1753,  0.0021,
          -0.2477],
         [ 0.0689, -0.0316,  0.0369, -0.0257,  0.0442,  0.0923,  0.1150,
           0.2653],
         [-0.1345, -0.3636, -0.2578, -0.3858,  0.0458, -0.1649, -0.1783,
          -0.2384],
         [ 0.2026,  0.1456,  0.1991,  0.1371,  0.1734,  0.1559,  0.1515,
           0.1263],
         [-0.0568, -0.0583, -0.0449, -0.2006, -0.1066, -0.2734, -0.1427,
          -0.2936],
         [ 0.0588, -0.1116, -0.1229, -0.2520,  0.2147,  0.0709,  0.1002,
           0.0622]]], device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([[[ 0.0120,  0.0449,  0.1118,  0.0654,  0.0717,  0.0507,  0.0780,
           0.0665],
         [-0.0111, -0.0095,  0.0003,  0.0027, -0.0360, -0.0460, -0.1143,
          -0.0730],
         [-0.0984

In [0]:
img = Variable(torch.Tensor(1, 8, 8).uniform_(-1, 1).float()).cuda()
img2 = Variable(torch.Tensor(1, 16, 16).fill_(0).float()).cuda()
img2[:,:,::2,::2] = img 
# every other element of img2 is equal to img1, so the convolutions should be equivalent
conv1 = nn.Conv2d(1,1, kernel_size=(3,3), padding=(1,1), dilation=(1,1), stride=(1,1)).cuda().float()
conv1d = deepcopy(conv1).cuda().float(); conv1d.padding = 2; conv1d.dilation = 2;
y1, y1d = img.clone(), img2.clone()
for i in range(2):
    y1, y1d = conv1(y1), conv1d(y1d)
    print(y1 == y1d[:,:,::2,::2])
    print('\ny1', y1.data.cpu().numpy(), '\n\ny1d', y1d[:,:,::2,::2].data.cpu().numpy())